In [58]:
import pandas as pd
import numpy as np
xmas = pd.read_csv('xmas.csv')

## the outputs are correct in positron but duplicated in the render file

#### Function 1: pluralize_gift()
#### it's vectorized!

In [59]:
def pluralize_gift(gift):
  """
  Returns plural of a list of nouns
  
  Parameters
  ----------
  gift: vector 
    nouns
    
  Return
  ------
  str
    Plural version
  """
    # If the input is a single string, convert it to a list for consistency
  if isinstance(gift, str):
      gift = [gift]

  pluralized_gift = []
    
  for i in gift:
      if "oo" in i:
          plugift = i.replace("oo", "ee")
      elif i.endswith("y"):
          plugift = i[:-1] + "ies"
      else:
          plugift = i + "s"
        
      pluralized_gift.append(plugift)
    
    # If the original input was a single string, return the pluralized version as a string
  if len(pluralized_gift) == 1:
      return pluralized_gift[0]
    
    # Otherwise, return the list of pluralized words
  return pluralized_gift

In [60]:
pluralize_gift(xmas['Gift.Item'])

['partridges',
 'doves',
 'hens',
 'birds',
 'rings',
 'geese',
 'swans',
 'maids',
 'ladies',
 'lords',
 'pipers',
 'drummers']

#### Function 2: make_phrase()

In [61]:
def make_phrase(num, item, verb, adjective, location):
  """
  Returns a full phrase for a specific day in '12 Days of Christmas' song.

  Parameters
  ----------
  num: int
      The day number
  item: str
      The gift item (e.g., "lord")
  verb: str
      The action (e.g., "a-leaping")
  adjective: str
      Optional adjective (e.g., "golden")
  location: str
      Optional location (e.g., "in a pear tree")

  Returns
  -------
  str
      The formatted phrase
  """
  
  ## Step 1: Replace NAs with blank strings
  verb = verb if pd.notna(verb) else ""
  adjective = adjective if pd.notna(adjective) else ""
  location = location if pd.notna(location) else ""
  
  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  num = int(num)
  if num > 1:
    item = pluralize_gift(item)
  
  ## Step 3: Figure out if a gift item starts with a vowel
  vowels = ('a', 'e', 'i', 'o', 'u')
  starts_with_vowel = item[0].lower() in vowels  
  
  ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
  number_words = ["Two", "Three", "Four", "Five", "Six", "Seven",
                  "Eight", "Nine", "Ten", "Eleven", "Twelve"]
  if num == 1:
    num_word = "an" if starts_with_vowel else "a"
  else:
    num_word = number_words[num - 2]

  ## Step 5: Put all of the pieces together into one string and return!
  components = [num_word, adjective, item, verb, location]
  # Filter out empty component before joining 
  phrase = " ".join(filter(None, components)).strip()
    
  return phrase

In [62]:
make_phrase(num = "10",
            item = "lord", 
            verb = "a-leaping", 
            adjective = "", 
            location = "")

'Ten lords a-leaping'

In [63]:
# Add Full Phrase column into the dataset
def add_full_phrase_column(dataset, phrase_col='Full.Phrase'):
    dataset[phrase_col] = list(
        map(lambda row: make_phrase(
            num=row['Day'], 
            item=row['Gift.Item'], 
            verb=row['Verb'], 
            adjective=row['Adjective'], 
            location=row['Location']
        ), dataset.to_dict('records'))
    )

#### Function 3: sing_day()

In [64]:
def sing_day(dataset, num, phrase_col):

    """
    Generate the lyrics for a specific day of the Christmas song.

    Parameters
    ----------
    dataset: DataFrame
        The dataset containing the gift phrases.
    num: int
        The day of Christmas to sing about.
    phrase_col: str
        The name of the column containing the gift phrases.

    Returns
    -------
    str
        The lyrics for the specified day.
    """

    if phrase_col not in dataset.columns:
       add_full_phrase_column(dataset, phrase_col)

    # Step 1: Setup the intro line
    num_word = dataset.loc[dataset['Day'] == num, 'Day.in.Words'].values[0]
    intro = f"On the {num_word} day of Christmas, my true love sent to me:"
    
    # Step 2: Sing the gift phrases
    gifts = list(map(lambda i: dataset.loc[dataset['Day'] == i, phrase_col].values[0] + ",", range(num, 0, -1)))
    if num > 1:
        gifts[-1] = "and " + gifts[-1].strip(",")
    else: 
        gifts[0] = gifts[0].strip(",")
    
    # Step 3: Put it all together and return the final verse
    full_song = intro + "\n" + "\n".join(gifts)
    full_song += "."
    
    return print(full_song)

In [65]:
sing_day(xmas, 2, "Full.Phrase")

On the second day of Christmas, my true love sent to me:
Two turtle doves,
and a partridge in a pear tree.
On the second day of Christmas, my true love sent to me:
Two turtle doves,
and a partridge in a pear tree.


In [66]:
# Use map to call sing_day for each day from 1 to 12
def entire_song_map(dataset, phrase_col):
    list(map(lambda day: (print() if day > 1 else None, sing_day(dataset, day, phrase_col)), range(1, 13)))
    
entire_song_map(xmas, "Full.Phrase")

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
Two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
Three french hens,
Two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
Four calling birds,
Three french hens,
Two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
Five golden rings,
Four calling birds,
Three french hens,
Two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
Six geese a-laying,
Five golden rings,
Four calling birds,
Three french hens,
Two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
Seven swans a-swimming,
Six geese a-laying,
Five golden rings,
Four calling birds,
Three french hens,
Two turtle doves,
and a 

#### Final output get rid of the white space, print a new line, print a separate line in between days, and add in commas and a period at the end

In [67]:
xmas2 = pd.read_csv('xmas_2.csv')

In [68]:
sing_day(xmas2, 3, "Full.Phrase")

On the third day of Christmas, my true love sent to me:
Three lost pens,
Two meal points,
and an email from Cal Poly.
On the third day of Christmas, my true love sent to me:
Three lost pens,
Two meal points,
and an email from Cal Poly.


In [69]:
entire_song_map(xmas2, "Full.Phrase")

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
Two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
Three lost pens,
Two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
Four course reviews,
Three lost pens,
Two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
Five practice exams,
Four course reviews,
Three lost pens,
Two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
Six graders grading,
Five practice exams,
Four course reviews,
Three lost pens,
Two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
Seven seniors stressing,
Six graders grading,
Five practice exams,
Four course reviews,
Three lost pens,
Two meal points,
and an email from Cal Poly.

On t